In [2]:
#!pip install gradio

In [3]:
#!pip install tensorflow

In [4]:
#!pip install --upgrade tensorflow

In [5]:
#!pip install --upgrade keras

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import PIL
import pathlib
from tensorflow import keras
#from tensorflow.keras import layers
from tensorflow.keras import layers, models

In [3]:
import gradio as gr

In [4]:
image_size = 100, 100
batch_size = 32

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
batch_size = 32
img_size = (100, 100)

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/dldata/dlimg',
     shuffle=True, batch_size=batch_size, image_size=img_size,
    validation_split=0.2, subset='training', seed=42)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/dldata/dlimg',
    shuffle=True, batch_size=batch_size, image_size=img_size,
    validation_split=0.2, subset='validation', seed=42)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/dldata/dlimg',
    shuffle=True, batch_size=batch_size, image_size=img_size)

Found 982 files belonging to 2 classes.
Using 786 files for training.
Found 982 files belonging to 2 classes.
Using 196 files for validation.
Found 982 files belonging to 2 classes.


In [7]:
class_names = train_dataset.class_names
print(class_names)

['Apple Braeburn', 'Banana']


In [ ]:
plt.figure(figsize = (15 ,15))
for images, labels in train_dataset.take(8):#take() returns images in given positional indices along the axis
    for i in range(14):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8")) #imshow() creates an image from a 2d numpy array
        # unit8 is an unsigned integer ranging from 0 to +255
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [9]:
num_classes=2

In [10]:
def cnn_model(lr=0.001):
    model = models.Sequential([
        layers.Rescaling(1./255),
        layers.Conv2D(32, 3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, 3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, 3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Adjust dropout rate
        layers.Dense(num_classes)
    ])

    model.compile(optimizer=keras.optimizers.Adam(lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [11]:
model = cnn_model()
epochs=10
history = model.fit(train_dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
25/25 [==============================] - 15s 522ms/step - loss: 0.3062 - accuracy: 0.9809 - val_loss: 8.0297 - val_accuracy: 0.4949
Epoch 2/10
25/25 [==============================] - 13s 520ms/step - loss: 0.1587 - accuracy: 0.9987 - val_loss: 8.1413 - val_accuracy: 0.4949
Epoch 3/10
25/25 [==============================] - 13s 507ms/step - loss: 0.1429 - accuracy: 1.0000 - val_loss: 8.1413 - val_accuracy: 0.4949
Epoch 4/10
25/25 [==============================] - 12s 469ms/step - loss: 0.1129 - accuracy: 1.0000 - val_loss: 8.1413 - val_accuracy: 0.4949
Epoch 5/10
25/25 [==============================] - 13s 523ms/step - loss: 0.0300 - accuracy: 1.0000 - val_loss: 8.1413 - val_accuracy: 0.4949
Epoch 6/10
25/25 [==============================] - 14s 574ms/step - loss: 0.0247 - accuracy: 1.0000 - val_loss: 8.1413 - val_accuracy: 0.4949
Epoch 7/10
25/25 [==============================] - 13s 510ms/step - loss: 0.0194 - accuracy: 0.9987 - val_loss: 8.1413 - val_accuracy: 0.4949

In [12]:
# printing the loss and accuracy
(loss, accuracy) = model.evaluate(valid_dataset)
print(loss)
print(accuracy)

model.save('model.h9')

7/7 [==============================] - 1s 95ms/step - loss: 4.0754 - accuracy: 0.7347
4.075390338897705
0.7346938848495483


In [16]:
def predict_input_image(img):
  img_4d=img.reshape(-1,100,100,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(2)}

In [ ]:
image = gr.inputs.Image(shape=(100,100))
label = gr.outputs.Label(num_top_classes=2)
iface = gr.Interface(fn=predict_input_image, inputs=image, outputs=label,interpretation='default')
iface.launch(debug=True,share=True)

<ipython-input-17-9b5669320a98>:1: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image = gr.inputs.Image(shape=(100,100))
<ipython-input-17-9b5669320a98>:1: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image = gr.inputs.Image(shape=(100,100))
<ipython-input-17-9b5669320a98>:2: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  label = gr.outputs.Label(num_top_classes=2)
<ipython-input-17-9b5669320a98>:2: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  label = gr.outputs.Label(num_top_classes=2)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://49e5d97fc68ec5102e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 20ms/step
